In [44]:
import sys

In [45]:
#!{sys.executable} -m pip install sqlalchemy

#### HOMEWORK 1) Task

In [1]:
# HOMEWORK 1)

from sqlalchemy.orm import declarative_base
#from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy import Column, Integer, String, ForeignKey

Base = declarative_base()

class Student(Base):
    __tablename__ = 'student' # звязати з таблиці

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    
    def __str__(self):
        return f'This is a student {self.name}. Age: {self.age}'
    
    def __repr__(self):
        return f'This is a student {self.name}. Age: {self.age}'

class Student_subject(Base):
    __tablename__ = 'student_subject' 
    id = Column(Integer, primary_key=True)
    student_id = Column(Integer, ForeignKey('student.id'))
    subject_id= Column(Integer, ForeignKey('subject.id'))
    def __str__(self):
        return f'This is a student_id {self.student_id}. subject_id: {self.subject_id}'
    
    def __repr__(self):
        return f'This is a student_id {self.student_id}. subject_id: {self.subject_id}'


class Subject(Base):
    __tablename__ = 'subject' 
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    def __str__(self):
        return f'This is a subject {self.name}'
    
    def __repr__(self):
        return f'This is a subject {self.name}'


In [2]:
from sqlalchemy import create_engine

DATABASE_URI = 'postgresql://{user}:{password}@{host}:{port}/{database}' # до якої бд підключаємось


engine = create_engine(
    DATABASE_URI.format(
        host='localhost',
        database='prjctr_lessons',
        user='tania',
        password='password',
        port=5432,
    )
)

In [3]:
Base.metadata.create_all(engine) 

In [4]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

session = Session()

In [5]:
first_student = session.query(Student).first()

In [6]:
first_student

This is a student Eddy. Age: 21

In [59]:
first_subj=session.query(Subject).first()

In [60]:
first_subj

This is a subject English

In [61]:
session.query(Student.name, Student.age + 5).all()

[('Bae', 23), ('Eddy', 26), ('Lily', 27), ('Jenny', 24)]

In [63]:
old_students = session.query(Student).filter(Student.age > 20).all()
# Select * from student where age > 20;
print(old_students)

[This is a student Eddy. Age: 21, This is a student Lily. Age: 22]


In [66]:
first_old_student = session.query(Student).filter(Student.age > 20).order_by(Student.id.desc()).first()
# Select * from student where age > 20 order by id desc limit 1;
print(first_old_student)

This is a student Lily. Age: 22


In [67]:
mature_students: list[Student] = (
    session.query(Student)
    .filter(Student.age.between(18, 20)) # >= and <=
    .all()
)
# Select * from student where age between 18 and 20

print(mature_students)

[This is a student Bae. Age: 18, This is a student Jenny. Age: 19]


In [68]:
### WHERE 

from sqlalchemy import and_

mature_students: list[Student] = (
    session.query(Student)
    .filter(
        and_(
            Student.age.between(18, 20),
            Student.name == 'Bae',
        )
    ).all()
)

# SELECT * from student where
# age between 18 and 20 and nama = 'Bae'
#

print(mature_students)

[This is a student Bae. Age: 18]


In [69]:
from sqlalchemy import or_

session.query(Student).filter(
    or_(
        Student.name == 'Bae',
        Student.name == 'Lila',
    )
).all()

# select * from student 
# where name = 'bae' or name = 'Lila'

[This is a student Bae. Age: 18]

In [70]:
session.query(Student).order_by(Student.name.desc()).limit(5).all()

# select * from student order by name desc limit 5;

[This is a student Lily. Age: 22,
 This is a student Jenny. Age: 19,
 This is a student Eddy. Age: 21,
 This is a student Bae. Age: 18]

In [71]:
session.query(Student).order_by(Student.name.desc()).limit(2).all()

# select * from student order by name desc limit 2;

[This is a student Lily. Age: 22, This is a student Jenny. Age: 19]

In [82]:
from sqlalchemy import and_, between


session.query(Student).filter(
        and_(
            or_(
                Student.age < 18,
                Student.age > 20,
            ),
            Student.id.between(1, 10)
        )
    ).order_by(Student.name.desc()).limit(10).all()



# Select * from student where (
    # id between 1 and 10 
    # and (
    #   age < 18 or age > 20
    #   )
# )
# order by name desc limit 10

[This is a student Lily. Age: 22, This is a student Eddy. Age: 21]

In [83]:
from random import randint

bae = session.query(Student).filter(Student.name == 'Bae').first()

print(bae)
print(bae.id)

This is a student Bae. Age: 18
1


In [84]:
# Перезапис 
bae.age = randint(18, 25)

print(bae)
print(bae.id)

This is a student Bae. Age: 23
1


In [86]:
print(bae.age)

23


In [87]:
session.commit() # Поки не зробите commit/rollback зміни на сервері не будуть дописані
# session.rollback()

In [88]:
bae

This is a student Bae. Age: 23

In [89]:
## Insert
new_student = Student(name='Lila super', age=25)

session.add(new_student)

In [90]:
session.query(Student).filter(
    and_(
        Student.name == 'Lila super',
        Student.age == 25,
    )
).all()

[This is a student Lila super. Age: 25]

In [91]:
session.commit()

In [92]:
from random import randint, choice # faker

random_names = ['Joe', 'Winston', 'Benedict', 'Andrea', 'Fillipe', 'Iria', 'Eric', 'Joan', 'Pablo', 'Brian', 'Timothy', 'Lee']

for _ in range(10):
    student = Student(name=choice(random_names), age=randint(18, 25))
    session.add(student)

session.commit()

In [93]:
session.query(Student).all()

[This is a student Eddy. Age: 21,
 This is a student Lily. Age: 22,
 This is a student Jenny. Age: 19,
 This is a student Bae. Age: 23,
 This is a student Lila super. Age: 25,
 This is a student Eric. Age: 20,
 This is a student Eric. Age: 23,
 This is a student Benedict. Age: 19,
 This is a student Joe. Age: 22,
 This is a student Pablo. Age: 23,
 This is a student Joe. Age: 23,
 This is a student Winston. Age: 24,
 This is a student Timothy. Age: 24,
 This is a student Brian. Age: 20,
 This is a student Fillipe. Age: 19]

In [94]:
last_student = session.query(Student).order_by(Student.id.desc()).first()

last_student

This is a student Fillipe. Age: 19

In [95]:
type(last_student)

__main__.Student

In [96]:
session.commit()

#### HOMEWORK 2) Task

In [116]:
# HOME WORK 2)
stud_eng=session.query(Student).join(Student_subject, Student.id==Student_subject.student_id).\
    join(Subject, Student_subject.subject_id==Subject.id).filter(Subject.name=='English').all()
for i in stud_eng:
    print(i)

This is a student Bae. Age: 23


In [117]:
session.commit()

#### HOMEWORK 3) Task

#### DISTINCT

```sql
1. SELECT DISTINCT name FROM student;
```

#### WHERE

```sql
2. SELECT name FROM student WHERE age < 20;
```


```sql
3. SELECT name FROM student WHERE NOT name = 'John';
```

```sql
4. SELECT name FROM student WHERE age < 20 AND age > 10;
5. SELECT name FROM student WHERE age < 20 OR name = 'John';
```


##### BETWEEN

```sql
6. SELECT name FROM student WHERE age BETWEEN 10 AND 20;
```

#### ORDER BY

```sql
7. SELECT name, age FROM student ORDER BY age;
8. SELECT name, age FROM student WHERE age BETWEEN 10 AND 20 ORDER BY age DESC;
```

In [138]:
from sqlalchemy import and_, or_

In [124]:
# optional 
# 1 SELECT DISTINCT name FROM student;
name=session.query(Student.name).distinct().all()
print(*name)

('Fillipe',) ('Lily',) ('Lila super',) ('Winston',) ('Jenny',) ('Eric',) ('Benedict',) ('Bae',) ('Timothy',) ('Joe',) ('Eddy',) ('Pablo',) ('Brian',)


In [127]:
# 2 SELECT name FROM student WHERE age < 20;
young_man=session.query(Student.name).filter(Student.age < 20).all()
young_man

[('Jenny',), ('Benedict',), ('Fillipe',)]

In [136]:
# 3 SELECT name FROM student WHERE NOT name = 'John';
student=session.query(Student.name).filter(Student.name!='John').all()
student

[('Eddy',),
 ('Lily',),
 ('Jenny',),
 ('Bae',),
 ('Lila super',),
 ('Eric',),
 ('Eric',),
 ('Benedict',),
 ('Joe',),
 ('Pablo',),
 ('Joe',),
 ('Winston',),
 ('Timothy',),
 ('Brian',),
 ('Fillipe',)]

In [141]:
# 4 SELECT name FROM student WHERE age < 20 AND age > 10;
student=session.query(Student.name).filter(
    and_( Student.age > 10,
         Student.age < 20)
    ).all()
student

[('Jenny',), ('Benedict',), ('Fillipe',)]

In [144]:
# 5 SELECT name FROM student WHERE age < 20 OR name = 'John';
student=session.query(Student.name).filter(
    or_( Student.age < 20,
         Student.name=='John')
    ).all()
student

[('Jenny',), ('Benedict',), ('Fillipe',)]

In [145]:
# 6 SELECT name FROM student WHERE age BETWEEN 10 AND 20;
student=session.query(Student.name).filter(Student.age.between(10, 20)).all()
student

[('Jenny',), ('Eric',), ('Benedict',), ('Brian',), ('Fillipe',)]

In [150]:
# 7 SELECT name, age FROM student ORDER BY age;
student=session.query(Student).order_by(Student.age.asc()).all()
print(*student, sep='\n')

This is a student Fillipe. Age: 19
This is a student Jenny. Age: 19
This is a student Benedict. Age: 19
This is a student Eric. Age: 20
This is a student Brian. Age: 20
This is a student Eddy. Age: 21
This is a student Joe. Age: 22
This is a student Lily. Age: 22
This is a student Pablo. Age: 23
This is a student Eric. Age: 23
This is a student Bae. Age: 23
This is a student Joe. Age: 23
This is a student Timothy. Age: 24
This is a student Winston. Age: 24
This is a student Lila super. Age: 25


In [151]:
# 8 SELECT name, age FROM student WHERE age BETWEEN 10 AND 20 ORDER BY age DESC;
student=session.query(Student).filter(Student.age.between(10,20)).order_by(Student.age.desc()).all()
print(*student, sep='\n')

This is a student Eric. Age: 20
This is a student Brian. Age: 20
This is a student Jenny. Age: 19
This is a student Benedict. Age: 19
This is a student Fillipe. Age: 19


#### JOIN

#### INNER JOIN

INNER JOIN - uses a comparison operator to match rows from two tables based on values in a common column that exists in **both** tables.

Simple join between two tables

```sql
1. 
SELECT *
FROM student
INNER JOIN student_subject
ON student.id = student_subject.student_id;
```

Select columns during the join
```sql
2. 
SELECT student.name, student_subject.subject_id
FROM student
INNER JOIN student_subject
ON student.id = student_subject.student_id;
```

Using alias
```sql
3. 
SELECT s.name AS student_name, ss.subject_id AS subject_id
FROM student AS s
INNER JOIN student_subject as ss
ON s.id = ss.student_id;
```

Join with three tables
```sql
4. 
SELECT *
FROM student
INNER JOIN student_subject ON student.id = student_subject.student_id
INNER JOIN subject ON subject.id = student_subject.subject_id;
```
```sql
5. 
SELECT * from (
    SELECT s.id, subject.id
    FROM student
    INNER JOIN subject ON subject.id = student.id;
)
```

```sql
6. 
SELECT *
FROM student
INNER JOIN subject ON subject.id = student.id;
```

#### OUTER JOIN

OUTER JOIN (left, right or full) - includes rows from **one or both** tables even if they don`t have matching values

```sql
7. 
SELECT *
FROM student
FULL JOIN student_subject
ON student.id = student_subject.student_id;
```

#### LEFT/RIGHT JOIN

CROSS JOIN - returns all rows from the left table with **all rows** from the right table. WHERE conditions should always be included. 

```sql
8. 
SELECT *
FROM student
LEFT JOIN student_subject
ON student.id = student_subject.student_id;
```

```sql
9. 
SELECT *
FROM student
RIGHT JOIN student_subject
ON student.id = student_subject.student_id;
```

In [8]:
# 1 SELECT *
# FROM student
# INNER JOIN student_subject
# ON student.id = student_subject.student_id;
studens=session.query(Student).join(Student_subject, Student.id==Student_subject.student_id).all()
studens

[This is a student Bae. Age: 23,
 This is a student Eddy. Age: 21,
 This is a student Lily. Age: 22,
 This is a student Jenny. Age: 19]

In [10]:
# 2 SELECT student.name, student_subject.subject_id
# FROM student
# INNER JOIN student_subject
# ON student.id = student_subject.student_id;
studens=session.query(Student.name, Student_subject.subject_id).\
    join(Student_subject, Student.id==Student_subject.student_id).all()
studens

[('Bae', 1), ('Eddy', 2), ('Lily', 3), ('Jenny', 4), ('Bae', 3)]

In [14]:
from sqlalchemy.orm import aliased

s = aliased(Student, name='s')
ss= aliased(Student_subject, name='ss')
su= aliased(Subject, name='su')


In [15]:
# 3 SELECT s.name AS student_name, ss.subject_id AS subject_id
# FROM student AS s
# INNER JOIN student_subject as ss
# ON s.id = ss.student_id;

studens=session.query(s.name, ss.subject_id).\
    join(ss, s.id==ss.student_id).all()
studens

[('Bae', 1), ('Eddy', 2), ('Lily', 3), ('Jenny', 4), ('Bae', 3)]

In [18]:
# 4 SELECT *
# FROM student
# INNER JOIN student_subject ON student.id = student_subject.student_id
# INNER JOIN subject ON subject.id = student_subject.subject_id;

students_classes=session.query(s).join(ss, s.id==ss.student_id).\
                    join(su, ss.subject_id==su.id).all()
students_classes


[This is a student Bae. Age: 23,
 This is a student Eddy. Age: 21,
 This is a student Lily. Age: 22,
 This is a student Jenny. Age: 19]

In [23]:
# 5 SELECT * from (
#   SELECT s.id, subject.id
#   FROM student
#   INNER JOIN subject ON subject.id = student.id;
# )
result=session.query(s.id, su.id).join(su, s.id==su.id).all()
result

[(1, 1), (2, 2), (3, 3), (4, 4)]

In [24]:
# 6 SELECT *
# FROM student
# INNER JOIN subject ON subject.id = student.id;

result=session.query(s).join(su, s.id==su.id).all()
result

[This is a student Bae. Age: 23,
 This is a student Eddy. Age: 21,
 This is a student Lily. Age: 22,
 This is a student Jenny. Age: 19]

In [34]:
# 7 SELECT *
# FROM student
# FULL JOIN student_subject
# ON student.id = student_subject.student_id;

result=session.query(s, ss).outerjoin(ss, s.id==ss.student_id).all()
result

[(This is a student Bae. Age: 23, This is a student_id 1. subject_id: 1),
 (This is a student Eddy. Age: 21, This is a student_id 2. subject_id: 2),
 (This is a student Lily. Age: 22, This is a student_id 3. subject_id: 3),
 (This is a student Jenny. Age: 19, This is a student_id 4. subject_id: 4),
 (This is a student Bae. Age: 23, This is a student_id 1. subject_id: 3),
 (This is a student Joe. Age: 23, None),
 (This is a student Winston. Age: 24, None),
 (This is a student Pablo. Age: 23, None),
 (This is a student Fillipe. Age: 19, None),
 (This is a student Timothy. Age: 24, None),
 (This is a student Lila super. Age: 25, None),
 (This is a student Benedict. Age: 19, None),
 (This is a student Eric. Age: 20, None),
 (This is a student Brian. Age: 20, None),
 (This is a student Joe. Age: 22, None),
 (This is a student Eric. Age: 23, None)]

In [37]:
# 8 SELECT *
# FROM student
# LEFT JOIN student_subject
# ON student.id = student_subject.student_id;

result=session.query(s, ss).join(ss, s.id==ss.student_id, isouter=True).all()
result

[(This is a student Bae. Age: 23, This is a student_id 1. subject_id: 1),
 (This is a student Eddy. Age: 21, This is a student_id 2. subject_id: 2),
 (This is a student Lily. Age: 22, This is a student_id 3. subject_id: 3),
 (This is a student Jenny. Age: 19, This is a student_id 4. subject_id: 4),
 (This is a student Bae. Age: 23, This is a student_id 1. subject_id: 3),
 (This is a student Joe. Age: 23, None),
 (This is a student Winston. Age: 24, None),
 (This is a student Pablo. Age: 23, None),
 (This is a student Fillipe. Age: 19, None),
 (This is a student Timothy. Age: 24, None),
 (This is a student Lila super. Age: 25, None),
 (This is a student Benedict. Age: 19, None),
 (This is a student Eric. Age: 20, None),
 (This is a student Brian. Age: 20, None),
 (This is a student Joe. Age: 22, None),
 (This is a student Eric. Age: 23, None)]

In [39]:
# 9 SELECT *
# FROM student
# RIGHT JOIN student_subject
# ON student.id = student_subject.student_id;

result=session.query(s, ss).join(ss, s.id==ss.student_id).all()
result

[(This is a student Bae. Age: 23, This is a student_id 1. subject_id: 1),
 (This is a student Eddy. Age: 21, This is a student_id 2. subject_id: 2),
 (This is a student Lily. Age: 22, This is a student_id 3. subject_id: 3),
 (This is a student Jenny. Age: 19, This is a student_id 4. subject_id: 4),
 (This is a student Bae. Age: 23, This is a student_id 1. subject_id: 3)]

#### AGGREGATE FUNCTIONS

1

```sql
SELECT COUNT(*) FROM student;
```

```sql
SELECT MIN(age) FROM student;
```

```sql
SELECT AVG(age) FROM student;

In [43]:
# 1 SELECT COUNT(*) FROM student;
count_=session.query(Student).count()
print(count_)

15


In [51]:
from sqlalchemy.sql import func  #(function avg is not in list(( )

In [59]:

# 2 SELECT MIN(age) FROM student;
min_=session.query(func.min(Student.age)).first()
print(*min_)

19


In [65]:
# 3 SELECT AVG(age) FROM student; 
avg_=session.query(func.sum(Student.age)/count_).first()
print(*avg_)

21.8000000000000000



#### UPDATE

Update one column in a row

```sql
UPDATE student SET age = 20 WHERE name = 'Bae';
```

Update multiple columns in a row
```sql
UPDATE student SET age = 20, name = 'John' WHERE name = 'Bae';
```

Update multiple rows
```sql
UPDATE student SET age = 20 WHERE age > 20;
```

Update based on existing values. Why this approach is bad?
```sql
UPDATE student SET age = age + 1 WHERE age > 10;
```



In [80]:
#1 UPDATE student SET age = 20 WHERE name = 'Bae';
session.query(Student).filter(Student.name=='Bae').\
    update({Student.age:20})

res=session.query(s).all()
print(*res, sep='\n')

This is a student Eddy. Age: 21
This is a student Lily. Age: 22
This is a student Jenny. Age: 19
This is a student Lila super. Age: 25
This is a student Eric. Age: 20
This is a student Eric. Age: 23
This is a student Benedict. Age: 19
This is a student Joe. Age: 22
This is a student Pablo. Age: 23
This is a student Joe. Age: 23
This is a student Winston. Age: 24
This is a student Timothy. Age: 24
This is a student Brian. Age: 20
This is a student Fillipe. Age: 19
This is a student Bae. Age: 20


In [81]:
session.rollback()

In [77]:
# UPDATE student SET age = 20, name = 'John' WHERE name = 'Bae';
session.query(Student).filter(Student.name=='Bae').update({Student.name:'John', Student.age:20})

1

In [78]:
res=session.query(s).all()
print(*res, sep='\n')

This is a student Eddy. Age: 21
This is a student Lily. Age: 22
This is a student Jenny. Age: 19
This is a student Lila super. Age: 25
This is a student Eric. Age: 20
This is a student Eric. Age: 23
This is a student Benedict. Age: 19
This is a student Joe. Age: 22
This is a student Pablo. Age: 23
This is a student Joe. Age: 23
This is a student Winston. Age: 24
This is a student Timothy. Age: 24
This is a student Brian. Age: 20
This is a student Fillipe. Age: 19
This is a student John. Age: 20


In [79]:
session.rollback()

In [82]:
# 3 UPDATE student SET age = 20 WHERE age > 20;
session.query(Student).filter(Student.age>20).update({Student.age: 20})

10

In [83]:
res=session.query(s).all()
print(*res, sep='\n')

This is a student Jenny. Age: 19
This is a student Eric. Age: 20
This is a student Benedict. Age: 19
This is a student Brian. Age: 20
This is a student Fillipe. Age: 19
This is a student Eddy. Age: 20
This is a student Lily. Age: 20
This is a student Bae. Age: 20
This is a student Lila super. Age: 20
This is a student Eric. Age: 20
This is a student Joe. Age: 20
This is a student Pablo. Age: 20
This is a student Joe. Age: 20
This is a student Winston. Age: 20
This is a student Timothy. Age: 20


In [84]:
session.rollback()

In [85]:
# 4 UPDATE student SET age = age + 1 WHERE age > 10;
session.query(Student).filter(Student.age>10).update({Student.age: Student.age+1})

15

In [86]:
res=session.query(s).all()
print(*res, sep='\n')

This is a student Eddy. Age: 22
This is a student Lily. Age: 23
This is a student Jenny. Age: 20
This is a student Bae. Age: 24
This is a student Lila super. Age: 26
This is a student Eric. Age: 21
This is a student Eric. Age: 24
This is a student Benedict. Age: 20
This is a student Joe. Age: 23
This is a student Pablo. Age: 24
This is a student Joe. Age: 24
This is a student Winston. Age: 25
This is a student Timothy. Age: 25
This is a student Brian. Age: 21
This is a student Fillipe. Age: 20


In [87]:
session.rollback()

In [93]:
session.commit()

In [94]:
session.close()